# DBMS Assignment

**Submitted BY : Subrat Kumar Nanda

In [67]:
import mysql.connector
import pandas as pd

In [95]:
import mysql.connector as mysql
myconn = mysql.connect(host='127.0.0.1',database='MY_BOOKS_SN', user='root', password='Admin')

In [16]:
# Test my Connection
myconn

**Get all Tables Name

In [19]:
tables = pd.read_sql_query('SHOW TABLES',myconn)
tables

,Tables_in_my_books_sn
0,author
1,book
2,book_author
3,book_language
4,cust_order
5,customer
6,customer_order_line
7,customer_paymet_details
8,distributor_order
9,distributor_order_line


In [47]:
tables[0:5]

,Tables_in_my_books_sn
0,author
1,book
2,book_author
3,book_language
4,cust_order


**Displaying Schema of all the Tables

In [45]:
for i in range(len(tables)):
    print(tables.loc[i, "Tables_in_my_books_sn"])
    print('_'*100)
    output = pd.read_sql_query('desc {}'.format(tables.loc[i, "Tables_in_my_books_sn"]), myconn)
    print(output, '\n')

author
____________________________________________________________________________________________________
         Field             Type Null  Key Default           Extra
0    author_id           b'int'   NO  PRI    None  auto_increment
1  author_name  b'varchar(400)'   NO         None                 

book
____________________________________________________________________________________________________
              Field             Type Null  Key              Default  \
0           book_id           b'int'   NO  PRI                 None   
1             title  b'varchar(400)'  YES                      None   
2              isbn   b'varchar(13)'   NO  UNI                 None   
3       language_id           b'int'  YES  MUL                 None   
4         num_pages           b'int'  YES                      None   
5        unit_price  b'decimal(5,2)'  YES                      None   
6  publication_date          b'date'  YES       b'(curdate() - 10)'   
7  publication_Cit

**Fetching Record from Warehouse Table

In [56]:
try:
    cursor = myconn.cursor();
    
    sql_select_Query = "select * from Warehouse WHERE book_id = 1 and store_id = 4"
    
    cursor.execute(sql_select_Query)
    # get all records
    records = cursor.fetchall()
    print("Total number of rows in Warehouse: ", cursor.rowcount)
    print("_"*25)
    print("\nPrinting each row")
    for row in records:
        print("Store_Id = ", row[0], )
        print("Book_Id = ", row[1])
        print("Available_Quantity = ", row[2] , "\n")

except mysql.connector.Error as e:
    print("Error reading data from MySQL table", e)

Total number of rows in Warehouse:  1
_________________________

Printing each row
Store_Id =  4
Book_Id =  1
Available_Quantity =  8 



**Calculating Price for a Book by its Id & Quantity 

In [70]:
sql_select_Query = "select calculate_order_price(17,2)" 
cursor.execute(sql_select_Query)
amount = cursor.fetchall()
print("Calculated Order Amount = {}".format(amount))

Calculated Order Amount = [(240.0,)]


**before_is_available Trigger

***Condition 1 - Invalid Quantity

In [72]:
try:
    mySql_insert_query = """INSERT INTO Customer_order_line  (order_id, book_id, quantity,price)
                                                  VALUES (%s, %s, %s, calculate_order_price(1,2)) """
    

    records_to_insert = [(257,1,-2)]

    cursor.executemany(mySql_insert_query, records_to_insert)
    # myconn.commit()
    print(cursor.rowcount, "Record inserted successfully into Customer_order_line table")

except mysql.connector.Error as error:
    print(" Failed to insert record into MySQL table : {}".format(error))

 Failed to insert record into MySQL table : 1644 (50001): PLEASE ENTER A VALID BOOK QUANTITY


***Condition 2 - Ord_Quantity > Available Quantity

***Get Book Availabity details

In [83]:
sql_select_Query = "select * from Warehouse WHERE book_id = 1 and store_id = 4" 
cursor.execute(sql_select_Query)
amount = cursor.fetchall()
#print("Available Book Quantity = {}".format(amount))
print("Available Book Quantity :")
print("Store_ID | Book_ID | Available_Quantity")
print("-"*25)
for row in amount:
    print("  {} \t |  {} \t   |\t{}\t".format(row[0],row[1],row[2]))

Available Book Quantity :
Store_ID | Book_ID | Available_Quantity
-------------------------
  4 	 |  1 	   |	8	


In [84]:
try:
    mySql_insert_query = """INSERT INTO Customer_order_line  (order_id, book_id, quantity,price)
                                                  VALUES (%s, %s, %s, calculate_order_price(1,2)) """
    

    records_to_insert = [(257,1,12)]

    cursor.executemany(mySql_insert_query, records_to_insert)
    # myconn.commit()
    print(cursor.rowcount, "Record inserted successfully into Customer_order_line table")

except mysql.connector.Error as error:
    print(" Failed to insert record into MySQL table : {}".format(error))

 Failed to insert record into MySQL table : 1644 (50002): NOT AVAILABLE - ORDER QUANTITY EXCEEDS AVAILABLE QUANTITY - AVAILABLE_BOOK_QTY = 8


***Condition 3 - Ord_Quantity > Available Quantity

In [85]:
try:
    mySql_insert_query = """INSERT INTO Customer_order_line  (order_id, book_id, quantity,price)
                                                  VALUES (%s, %s, %s, calculate_order_price(1,2)) """
    

    records_to_insert = [(257,1,2)]

    cursor.executemany(mySql_insert_query, records_to_insert)
    # myconn.commit()
    print(cursor.rowcount, "Record inserted successfully into Customer_order_line table")

except mysql.connector.Error as error:
    print(" Failed to insert record into MySQL table : {}".format(error))

1 Record inserted successfully into Customer_order_line table


In [87]:
myconn.commit()

In [99]:
#cursor = myconn.cursor();
sql_select_Query = "select * from Warehouse WHERE book_id = 1 and store_id = 4;" 
cursor.execute(sql_select_Query)
amount = cursor.fetchall()
#print("Available Book Quantity = {}".format(amount))
print("Available Book Quantity :")
print("Store_ID | Book_ID | Available_Quantity")
print("-"*25)
for row in amount:
    print("  {} \t |  {} \t   |\t{}\t".format(row[0],row[1],row[2]))

Available Book Quantity :
Store_ID | Book_ID | Available_Quantity
-------------------------
  4 	 |  1 	   |	6	


***Write PYTHON codes that connects to this database and use SQL commands to select and
retrieve information about availability of books identified by title, author, publisher or year
of publication

In [101]:
sql_select_Query = """SELECT B.BOOK_ID,B.TITLE,B.publication_date,A.author_NAME, P.publisher_name,S.store_name, W.available_quantity FROM BOOK B INNER JOIN Publisher P ON B.publisher_id = P.publisher_id 
INNER JOIN Book_Author BA ON B.BOOK_ID = BA.BOOK_ID   
INNER JOIN AUTHOR A ON A.author_id = BA.author_id     
INNER JOIN Warehouse W ON W.BOOK_ID = B.BOOK_ID       
INNER JOIN STORE S ON S.STORE_ID = W.STORE_ID          
WHERE B.BOOK_ID IN (1,2,3,4,5,6,7,8,9,10);"""          

cursor.execute(sql_select_Query)
cursor.fetchall()


[(1,
  "The World's First Love: Mary  Mother of God",
  datetime.date(1996, 9, 1),
  'Fulton J. Sheen',
  'Ignatius Press',
  'Distributor 1',
  40),
 (1,
  "The World's First Love: Mary  Mother of God",
  datetime.date(1996, 9, 1),
  'Fulton J. Sheen',
  'Ignatius Press',
  'Distributor 4',
  6),
 (2,
  'The Illuminati',
  datetime.date(2004, 10, 4),
  'Larry Burkett',
  'Thomas Nelson',
  'Distributor 1',
  40),
 (2,
  'The Illuminati',
  datetime.date(2004, 10, 4),
  'Larry Burkett',
  'Thomas Nelson',
  'Distributor 4',
  32),
 (3,
  'The Servant Leader',
  datetime.date(2003, 3, 11),
  'Kenneth H. Blanchard',
  'Thomas Nelson',
  'Distributor 1',
  40),
 (3,
  'The Servant Leader',
  datetime.date(2003, 3, 11),
  'Kenneth H. Blanchard',
  'Thomas Nelson',
  'Distributor 5',
  32),
 (4,
  'What Life Was Like in the Jewel in the Crown: British India  AD 1600-1905',
  datetime.date(1999, 9, 1),
  'Time-Life Books',
  'Time Life Medical',
  'Distributor 1',
  40),
 (4,
  'What Life Wa

In [102]:
my_result_01 = pd.read_sql_query(sql_select_Query,myconn)
my_result_01

,BOOK_ID,TITLE,publication_date,author_NAME,publisher_name,store_name,available_quantity
0,1,The World's First Love: Mary Mother of God,1996-09-01,Fulton J. Sheen,Ignatius Press,Distributor 1,40
1,1,The World's First Love: Mary Mother of God,1996-09-01,Fulton J. Sheen,Ignatius Press,Distributor 4,6
2,2,The Illuminati,2004-10-04,Larry Burkett,Thomas Nelson,Distributor 1,40
3,2,The Illuminati,2004-10-04,Larry Burkett,Thomas Nelson,Distributor 4,32
4,3,The Servant Leader,2003-03-11,Kenneth H. Blanchard,Thomas Nelson,Distributor 1,40
5,3,The Servant Leader,2003-03-11,Kenneth H. Blanchard,Thomas Nelson,Distributor 5,32
6,4,What Life Was Like in the Jewel in the Crown: ...,1999-09-01,Time-Life Books,Time Life Medical,Distributor 1,40
7,4,What Life Was Like in the Jewel in the Crown: ...,1999-09-01,Time-Life Books,Time Life Medical,Distributor 5,32
8,5,Cliffs Notes on Aristophanes' Lysistrata The ...,1983-12-29,W. John Campbell,Cliffs Notes,Distributor 1,40
9,5,Cliffs Notes on Aristophanes' Lysistrata The ...,1983-12-29,W. John Campbell,Cliffs Notes,Distributor 6,32


In [103]:
if myconn.is_connected():
    cursor.close()
    myconn.close()
    print("MySQL connection is closed")

MySQL connection is closed
